In [ ]:
# https://github.com/kserve/kserve/tree/master/docs

### iris-example.yaml
```
apiVersion: "serving.kserve.io/v1beta1"
kind: "InferenceService"
metadata:
  annotations:
    sidecar.istio.io/inject: "false"
  name: "sklearn-iris"
spec:
  predictor:
    sklearn:
      image: "kserve/sklearnserver:v0.9.0"
      storageUri: "gs://kfserving-examples/models/sklearn/1.0/model"
```
- example의 model은 kserve8.0 이상에서 동작하도록 만들어져 있기 때문에 image를 따로 불러와야 한다. 

### iris-request.yaml
```
{
    "instances": [
      [6.8,  2.8,  4.8,  1.4],
      [6.0,  3.4,  4.5,  1.6]
    ]
}
```

In [1]:
!pip install kserve

In [3]:
import warnings
warnings.filterwarnings("ignore")

In [4]:
from kserve import (
    constants,
    KServeClient,
    V1beta1InferenceService,
    V1beta1InferenceServiceSpec,
    V1beta1PredictorSpec,
    V1beta1SKLearnSpec,
    V1beta1ModelSpec,
    V1beta1ModelFormat,
    V1beta1TFServingSpec,
    utils
)
from kubernetes import client 

In [5]:
namespace = utils.get_default_target_namespace()
service_name="sklearn-iris-python"

api_version = constants.KSERVE_GROUP + '/' + constants.KSERVE_V1BETA1_VERSION
storage = 'gs://kfserving-examples/models/sklearn/1.0/model'
image = "kserve/sklearnserver:v0.9.0"
isvc = V1beta1InferenceService(api_version=api_version,
                               kind=constants.KSERVE_KIND,
                               metadata=client.V1ObjectMeta(name=service_name, namespace=namespace, annotations={'sidecar.istio.io/inject':'false'}),
                               spec=V1beta1InferenceServiceSpec(
                                   predictor=V1beta1PredictorSpec(
                                       sklearn=(V1beta1SKLearnSpec(
                                           image = image,
                                            storage_uri = storage))))
)

In [6]:
KServe = KServeClient()
KServe.create(isvc)

{'apiVersion': 'serving.kserve.io/v1beta1',
 'kind': 'InferenceService',
 'metadata': {'annotations': {'sidecar.istio.io/inject': 'false'},
  'creationTimestamp': '2024-03-13T09:06:39Z',
  'generation': 1,
  'managedFields': [{'apiVersion': 'serving.kserve.io/v1beta1',
    'fieldsType': 'FieldsV1',
    'fieldsV1': {'f:metadata': {'f:annotations': {'.': {},
       'f:sidecar.istio.io/inject': {}}},
     'f:spec': {'.': {},
      'f:predictor': {'.': {},
       'f:sklearn': {'.': {},
        'f:image': {},
        'f:name': {},
        'f:storageUri': {}}}}},
    'manager': 'OpenAPI-Generator',
    'operation': 'Update',
    'time': '2024-03-13T09:06:36Z'}],
  'name': 'sklearn-iris-python',
  'namespace': 'user2',
  'resourceVersion': '30632907',
  'uid': '733a4833-f676-48eb-8820-7315a32462ac'},
 'spec': {'predictor': {'model': {'image': 'kserve/sklearnserver:v0.9.0',
    'modelFormat': {'name': 'sklearn'},
    'name': '',
    'resources': {},
    'storageUri': 'gs://kfserving-examples/m

In [7]:
KServe.get(service_name, namespace=namespace, watch=True, timeout_seconds=120)
isvc_resp=KServe.get(service_name, namespace=namespace)

NAME                 READY      PREV    LATEST  URL
sklearn-iris-python  Unknown       0       100
NAME                 READY      PREV    LATEST  URL
sklearn-iris-python  Unknown       0       100
NAME                 READY      PREV    LATEST  URL
sklearn-iris-python  Unknown       0       100
NAME                 READY      PREV    LATEST  URL
sklearn-iris-python  Unknown       0       100
NAME                 READY      PREV    LATEST  URL
sklearn-iris-python  True          0       100  https://sklearn-iris-python-predictor.user2.ezua.ksi.local


In [8]:
isvc_resp

{'apiVersion': 'serving.kserve.io/v1beta1',
 'kind': 'InferenceService',
 'metadata': {'annotations': {'sidecar.istio.io/inject': 'false'},
  'creationTimestamp': '2024-03-13T09:06:39Z',
  'finalizers': ['inferenceservice.finalizers'],
  'generation': 1,
  'managedFields': [{'apiVersion': 'serving.kserve.io/v1beta1',
    'fieldsType': 'FieldsV1',
    'fieldsV1': {'f:metadata': {'f:annotations': {'.': {},
       'f:sidecar.istio.io/inject': {}}},
     'f:spec': {'.': {},
      'f:predictor': {'.': {},
       'f:sklearn': {'.': {},
        'f:image': {},
        'f:name': {},
        'f:storageUri': {}}}}},
    'manager': 'OpenAPI-Generator',
    'operation': 'Update',
    'time': '2024-03-13T09:06:36Z'},
   {'apiVersion': 'serving.kserve.io/v1beta1',
    'fieldsType': 'FieldsV1',
    'fieldsV1': {'f:metadata': {'f:finalizers': {'.': {},
       'v:"inferenceservice.finalizers"': {}}}},
    'manager': 'manager',
    'operation': 'Update',
    'time': '2024-03-13T09:06:39Z'},
   {'apiVersi

In [9]:
sklearn_iris_input={
    "instances": [
      [6.8,  2.8,  4.8,  1.4],
      [6.0,  3.4,  4.5,  1.6]
    ]
}

In [11]:
import requests
isvc_url = isvc_resp['status']['address']['url']
print(isvc_url)
rsvc_url = isvc_url +"/v1/models/" + service_name + ":predict"
response = requests.post(rsvc_url, json=sklearn_iris_input)
print(response.text)

http://sklearn-iris-python-predictor.user2.svc.cluster.local
{"predictions": [1, 1]}


In [ ]:
# curl을 이용한 요청

In [12]:
import requests
import kfp

HOST = "https://kubeflow.ezua.ksi.local/"
USERNAME = "user2@example.com"
PASSWORD = "user23456"

session = requests.Session()
response = session.get(HOST, verify=False) #ssl 접속시 verify=False 

headers = {
    "Content-Type": "application/x-www-form-urlencoded",
}

data = {"login": USERNAME, "password": PASSWORD}
session.post(response.url, headers=headers, data=data)
# session_cookie = session.cookies.get_dict()["authservice_session"]
session_cookie = session.cookies.get_dict()
print(session_cookie)

{'_oauth2_proxy_csrf_-4sxIn-j': 'laWRsHk4B8aQCh44eHR15yVMp39xzins7Fkec8kmF03TXlxhH7I9GG6qO-CxUhmDXBS7teXQ5PFHtNuG4X4JOqwwQpkaB2RwctbsCoPcwFPY9cIdRNj_Xik=|1710321170|h7SYiedfJFy7XlFGfMyY11UHU0kIwq8MIeWHciqKU6o=', 'AUTH_SESSION_ID': '85ba775a-9196-4229-ba6e-87c024377069.keycloak-0-19908', 'AUTH_SESSION_ID_LEGACY': '85ba775a-9196-4229-ba6e-87c024377069.keycloak-0-19908', 'KC_RESTART': 'eyJhbGciOiJIUzI1NiIsInR5cCIgOiAiSldUIiwia2lkIiA6ICIxOThhYjZkOS01M2M1LTQ5NTQtYTg4Ni01YWI2YzI0MWI0NzEifQ.eyJjaWQiOiJ1YSIsInB0eSI6Im9wZW5pZC1jb25uZWN0IiwicnVyaSI6Imh0dHBzOi8va3ViZWZsb3cuZXp1YS5rc2kubG9jYWwvb2F1dGgyL2NhbGxiYWNrIiwiYWN0IjoiQVVUSEVOVElDQVRFIiwibm90ZXMiOnsic2NvcGUiOiJvcGVuaWQgZW1haWwgcHJvZmlsZSBvZmZsaW5lX2FjY2VzcyIsImlzcyI6Imh0dHBzOi8va2V5Y2xvYWsuZXp1YS5rc2kubG9jYWwvcmVhbG1zL1VBIiwicmVzcG9uc2VfdHlwZSI6ImNvZGUiLCJyZWRpcmVjdF91cmkiOiJodHRwczovL2t1YmVmbG93LmV6dWEua3NpLmxvY2FsL29hdXRoMi9jYWxsYmFjayIsInN0YXRlIjoiLTRzeEluLWpTbWRtREpHMkplUmxIQ2xOcDRjaG1STkNzTEN6eXZ0djBnNDpodHRwczovL3Rva2VuLXNlcnZpY2UuZXp

In [17]:
import json
headers = {"Host": "sklearn-iris-python-predictor.user2.ezua.ksi.local"}
res = session.post("https://kubeflow.ezua.ksi.local/v1/models/sklearn-iris-python:predict", headers=headers, cookies=session_cookie, data=json.dumps(sklearn_iris_input))
print(res.json())

JSONDecodeError: Expecting value: line 1 column 1 (char 0)